**INSTALL AND IMPORT LIBRARIES**

In [1]:
!pip install scikit-learn pandas numpy joblib

import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

**LOAD DATA+MODEL**

In [2]:
df = pd.read_csv('leads_scoring.csv', dtype=str, low_memory=False)
tfidf = joblib.load('lead_tfidf.pkl')

**EXTRACT NAME FROM OTHER CSV FOR NAME RECS**

In [3]:
import pandas as pd

df = pd.read_csv('leads_scoring.csv', dtype=str, low_memory=False)
df_cleaned = pd.read_csv('leads_cleaned.csv', dtype=str, low_memory=False)

df_name_info = df_cleaned[['permalink']].copy()
df_name_info['name'] = df_name_info['permalink']
df_name_info = df_name_info.drop(columns=['permalink'])

df_merged = pd.concat([df_name_info, df], axis=1)

cols = ['name'] + [col for col in df_merged.columns if col != 'name']
df_merged = df_merged[cols]

df_merged['name'] = df_merged['name'].str.replace('/organization/', '', regex=False)

df_merged.to_csv('leads_recommender.csv', index=False)

In [8]:
df = pd.read_csv('leads_recommender.csv', dtype=str, low_memory=False)

**TRANSFORM TEXT TO TF-IDF**

In [9]:
if any(col.startswith('tfidf_') for col in df.columns):
    tfidf_features = [col for col in df.columns if col.startswith('tfidf_')]
    tfidf_matrix = df[tfidf_features].values
else:
    tfidf_matrix = tfidf.transform(df['text_features'].astype(str)).toarray()

**COSINE SIMILARITY**

In [16]:
company_idx = 35
company_name = df.iloc[company_idx]['name'] if 'name' in df.columns else f"Company {company_idx}"

sims = cosine_similarity([tfidf_matrix[company_idx]], tfidf_matrix)[0]

top_indices = np.argsort(sims)[::-1][1:6]
similar_companies = df.iloc[top_indices]

print(f"Selected company: {company_name}")
print("Top 5 most similar companies:")
display_cols = ['name', 'market', 'city', 'country_code'] if 'name' in df.columns else df.columns
print(similar_companies[display_cols].reset_index(drop=True).to_string(index=False))

Selected company: 10x10-room
Top 5 most similar companies:
              name   market          city country_code
      sai-medisoft software        durham          usa
        feed-media software san francisco          usa
          sagefire software       boulder          usa
saffron-technology software         other          usa
           feedzai software     san mateo          usa
